In [1]:
import os, sys
path_name= os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd())))
sys.path.append(path_name)
print(path_name)

C:\Nextop\nextop-engine\nextop_engine


In [2]:
from _element import feature_control as ft_c
from _element.data_control import Data, DataRestruction, DataAddition
from _element import varr
from _element import calculations as calc
from _element.result_control import Result

from _alg.prophet import Prophet_timeseries
from _alg.arima import Arima_timeseries

from _evaluation.cross_validation import Cross_Validation

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [21]:
'\t ' in '\tabc\t'

False

In [30]:
data_return= Data()
for full_name in ft_c.dir_list(varr.PJ_DIR+ '\\_element\\data\\private_return\\Rawdata', '.csv'):
    INPUT_FILENAME= full_name.split('\\')[-1]
    print(INPUT_FILENAME)
    df_raw= pd.read_csv(full_name, engine= 'python', encoding= 'euc-kr')
    data_return.setRawData(df_raw, rawdata_setting= True)
    restruct= DataRestruction(data_return)
    restruct.groupbyObject(idx_col= '일자', ft_col= '유형', val_col= '수량', y_sum= True, idx_to_datetime= True)
    restruct.data[frozenset({'raw'})].columns= restruct.data[frozenset({'raw'})].columns.map(lambda x: int(x) if type(x)==float else x)
    restruct.commitRestructedData()
    merge_data= pd.concat([merge_data, data_return.data], axis= 0)

2010.csv
     회사코드        일자      유형   수량
0  297187  20100101  1815.0   26
1  296285  20100101  1815.0   30
2  289843  20100101  1815.0  200
3  284064  20100101  1261.0   15
4  230773  20100101  1815.0    2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48623 entries, 0 to 48622
Data columns (total 4 columns):
회사코드    48623 non-null int64
일자      48623 non-null int64
유형      48622 non-null float64
수량      48623 non-null int64
dtypes: float64(1), int64(3)
memory usage: 1.5 MB
None
          ds  1025   1041  1091  1127  1216  1242    1261   1355   1373  \
0 2010-01-01   0.0    5.0   0.0   0.0   0.0   0.0    15.0    0.0    0.0   
1 2010-01-02  63.0  157.0  38.0   0.0   2.0   5.0  1429.0    0.0   45.0   
2 2010-01-03   0.0    0.0   0.0   0.0   0.0   0.0     0.0    0.0    0.0   
3 2010-01-04  27.0  109.0   0.0   0.0   0.0   3.0  1355.0  149.0  261.0   
4 2010-01-05  50.0  183.0   0.0   0.0   2.0  14.0  1816.0    0.0    6.0   

    ...       1692  1745   1797  1800     1815  1817    1853 

          ds  1025  1032   1041  1057  1091  1119  1127  1163  1216   ...     \
0 2015-01-01   0.0   0.0   16.0   0.0   0.0   0.0   0.0   0.0   0.0   ...      
1 2015-01-02   0.0   0.0  190.0   0.0  67.0   0.0   0.0   0.0   0.0   ...      
2 2015-01-03   0.0   0.0  254.0   0.0   0.0   0.0   0.0   0.0   0.0   ...      
3 2015-01-04   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0   ...      
4 2015-01-05   0.0   0.0  169.0   0.0   0.0   0.0   0.0   0.0   0.0   ...      

   1745  1754  1797  1800     1815  1817   1853   1878  1891    y_sum  
0   0.0   0.0   0.0   0.0   7288.0   0.0    0.0    0.0   0.0   8813.0  
1   0.0   0.0   0.0   0.0  19425.0   0.0  141.0  196.0   0.0  22871.0  
2   0.0   0.0   0.0   0.0  14145.0   0.0  285.0   77.0   0.0  17759.0  
3   0.0   0.0   0.0   0.0    334.0   0.0    0.0    0.0   0.0    334.0  
4   0.0   0.0   0.0   0.0  22187.0   0.0  157.0   56.0   7.0  27428.0  

[5 rows x 31 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0

In [4]:
INPUT_FILENAME= 'KPP_계약처_입고출고(2010-2014).xlsx'
df_raw= ft_c.xlsx_opener(INPUT_FILENAME)

In [5]:
df_raw.head()

,타입,코드,발송일,유형코드,수량
0,입고,298137,20100101,1815.0,47
1,입고,298115,20100101,1815.0,77
2,입고,298108,20100101,1815.0,4
3,입고,298053,20100101,1815.0,56
4,입고,297994,20100101,1815.0,51


In [6]:
df_in= df_raw[df_raw['타입']== '입고']

In [7]:
df_out= df_raw[df_raw['타입']== '출하']

In [8]:
data_out= Data()

In [9]:
data_out.setRawData(df_out)

     타입      코드       발송일    유형코드   수량
928  출하  298152  20100101  1815.0    3
929  출하  298152  20100101  1692.0    7
930  출하  298141  20100101  1815.0   27
931  출하  298122  20100101  1692.0    3
932  출하  298115  20100101  1815.0  768
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3922661 entries, 928 to 327922
Data columns (total 5 columns):
타입      object
코드      int64
발송일     int64
유형코드    float64
수량      int64
dtypes: float64(1), int64(3), object(1)
memory usage: 179.6+ MB
None


In [10]:
restruct= DataRestruction(data_out)

In [11]:
restruct.groupbyObject(idx_col= '발송일', ft_col= '유형코드', val_col= '수량', y_sum= True, idx_to_datetime= True)

In [12]:
restruct.data[frozenset({'raw'})].columns= restruct.data[frozenset({'raw'})].columns.map(lambda x: int(x) if type(x)==float else x)

In [13]:
restruct.commitRestructedData()

          ds  1025  1032    1041  1057   1091  1101  1111  1117  1127  \
0 2010-01-01   0.0   0.0  1299.0   0.0   92.0   0.0   0.0   0.0   0.0   
1 2010-01-02  10.0   0.0   455.0   0.0   80.0   0.0   0.0   0.0   0.0   
2 2010-01-03   0.0   0.0   132.0   0.0    4.0   0.0   0.0   0.0   0.0   
3 2010-01-04   1.0   0.0  1952.0   0.0  356.0   0.0   0.0   0.0   0.0   
4 2010-01-05   0.0   0.0  2077.0   0.0  367.0   0.0   0.0   0.0   2.0   

     ...     1797  1800      1815  1817  1818  1852    1853    1878  1891  \
0    ...     96.0   0.0  647711.0   0.0   0.0   0.0   380.0  2042.0  40.0   
1    ...      5.0   0.0   38071.0  10.0   0.0   0.0   496.0  3794.0   0.0   
2    ...      0.0   0.0    6840.0   0.0   0.0   0.0   100.0    27.0   0.0   
3    ...     37.0   0.0   42383.0  34.0   0.0   0.0  4317.0  8221.0   0.0   
4    ...      7.0   0.0   58017.0  27.0   0.0   0.0  4321.0  6588.0   6.0   

      y_sum  
0  667170.0  
1   51089.0  
2    8901.0  
3   74659.0  
4   90428.0  

[5 rows x 39 

In [14]:
df_raw.to_pickle('C:\\Nextop\\nextop-engine\\nextop_engine\\_element\\data\\private_IO\\KPP_계약처_입고출고(2010-2014).pickle')

In [15]:
data_out.data[frozenset({'raw'})].to_pickle('C:\\Nextop\\nextop-engine\\nextop_engine\\_element\\data\\private_IO\\KPP_계약처_입고출고(2010-2014)_daily_obj.pickle')

In [16]:
INPUT_FILENAME= 'KPP_계약처_입고출고(2015-2017).xlsx'
df_raw2015= ft_c.xlsx_opener(INPUT_FILENAME)

In [17]:
df_raw2015.head()

,타입,코드,발송일,유형코드,수량
0,입고,298152,20150101,1815.0,40
1,입고,298045,20150101,1815.0,83
2,입고,298031,20150101,1891.0,5
3,입고,298023,20150101,1815.0,1145
4,입고,298009,20150101,1853.0,1141


In [18]:
df_in2015= df_raw2015[df_raw2015['타입']== '입고']

In [19]:
df_out2015= df_raw2015[df_raw2015['타입']== '출하']

In [20]:
data_out2015= Data()

In [21]:
data_out2015.setRawData(df_out2015)

        타입      코드       발송일    유형코드   수량
270062  출하  298152  20150101  1692.0  149
270063  출하  298137  20150101  1815.0   19
270064  출하  298064  20150101  1878.0  752
270065  출하  298045  20150101  1815.0   32
270066  출하  298038  20150101  1815.0   19
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3181370 entries, 270062 to 605881
Data columns (total 5 columns):
타입      object
코드      int64
발송일     int64
유형코드    float64
수량      int64
dtypes: float64(1), int64(3), object(1)
memory usage: 145.6+ MB
None


In [22]:
restruct2015= DataRestruction(data_out2015)

In [23]:
restruct2015.groupbyObject(idx_col= '발송일', ft_col= '유형코드', val_col= '수량', y_sum= True, idx_to_datetime= True)

In [24]:
restruct2015.data[frozenset({'raw'})].columns= restruct2015.data[frozenset({'raw'})].columns.map(lambda x: int(x) if type(x)==float else x)

In [25]:
restruct2015.commitRestructedData()

          ds  1025  1032    1041  1057  1091  1101  1119  1127  1216  \
0 2015-01-01   0.0   0.0  2115.0   0.0  29.0   0.0   0.0   0.0   0.0   
1 2015-01-02   0.0   0.0  4339.0   0.0  75.0   0.0   0.0   0.0   0.0   
2 2015-01-03   0.0   0.0  2403.0  41.0  21.0   0.0   0.0   0.0   0.0   
3 2015-01-04   0.0   0.0    21.0   0.0  53.0   0.0   0.0   0.0   0.0   
4 2015-01-05   0.0   0.0  4644.0   0.0  85.0   0.0   0.0   0.0   0.0   

     ...      1754  1797  1800      1815  1817  1818    1853     1878   1891  \
0    ...     410.0   0.0   0.0  144551.0   0.0   0.0  1161.0   5208.0    0.0   
1    ...     925.0   0.0   0.0   87737.0  78.0   0.0  4891.0   9508.0   15.0   
2    ...       3.0   0.0   0.0   56624.0   0.0   0.0  2296.0   1802.0    0.0   
3    ...      32.0   0.0   0.0   16503.0   0.0   0.0   331.0     57.0    0.0   
4    ...     433.0   0.0   0.0   90064.0  39.0   0.0  6654.0  10739.0  211.0   

      y_sum  
0  174168.0  
1  135573.0  
2   75269.0  
3   19952.0  
4  141355.0  

[

In [26]:
df_raw2015.to_pickle('C:\\Nextop\\nextop-engine\\nextop_engine\\_element\\data\\private_IO\\KPP_계약처_입고출고(2015-2017).pickle')
data_out2015.data[frozenset({'raw'})].to_pickle('C:\\Nextop\\nextop-engine\\nextop_engine\\_element\\data\\private_IO\\KPP_계약처_입고출고(2015-2017)_daily_obj.pickle')